In [1]:
import requests
import bs4
import re
import datetime as dt
import pandas as pd
import pdb

In [2]:
def print_news(term):
    """
    Prints the headlines from a google search of term. Sorts by most recent.
    """
    term = term.replace(' ', '+')
    url = 'https://www.google.com/search?q={}&source=lnms&tbm=nws'.format(term)

    r = requests.get(url)
    assert r.status_code == 200

    soup = bs4.BeautifulSoup(r.text, 'lxml')
    headlines = soup.find_all('div', {'class':'g'})

    headline_list = []
    source_list = []
    time_list = []

    for headline in headlines:
        link = headline.find('h3')
        source = headline.find('div', {'class':'slp'})

        headline = link.text
        source_time = source.text

        # Record headline
        headline_list.append(headline)

        # Record source
        source_text = re.search(r'^(.*) - ', source_time).group(1)
        source_list.append(source_text)

        # Convert dates
        time_string = re.search(r' - (.*)$', source_time).group(1)

        # Check if in format x days ago
        days_ago_format = re.search(r'^(\d+) (.*) ago$', time_string)

        if days_ago_format:
            if days_ago_format.group(2) == 'days':
                
                days_ago = int(days_ago_format.group(1))
                timestamp = dt.datetime.today() - dt.timedelta(days_ago)
            else:
                timestamp = dt.datetime.today()
        else:
            date_string = re.search(r'^\d{1,2} \D{3} \d{4}', time_string)
            if date_string:
                timestamp = dt.datetime.strptime(date_string.group(), '%d %b %Y')
            else:
                timestamp = None

        time_list.append(timestamp)

    news_list = zip(time_list, headline_list, source_list)
    for time, headline, source in sorted(news_list, reverse=True):
        print(time.date())
        print(headline)
        print(source)
        print()

In [4]:
term = 'Charlotte Hornets'

print_news(term)

2019-01-09
Rumor: Charlotte Hornets interested in trade for Kevin Love
Swarm and Sting

2019-01-09
Kemba Walker: I trust Hornets to build appealing roster
ESPN

2019-01-09
Hornets' Kemba Walker has quiet night in win over Suns, until crazy ...
Charlotte Observer

2019-01-09
Hornets questions: On Devonte Graham's minutes, Kemba the All ...
Charlotte Observer

2019-01-09
Hornets Head to LA to Take on Clippers
Hornets.com

2019-01-09
Gerald Henderson joining Charlotte Hornets broadcast team
Swarm and Sting

2019-01-09
Charlotte Hornets: Mid-season grades for Charlotte's rookies
Swarm and Sting

2019-01-09
Charlotte Hornets look to even their record vs Clippers
Swarm and Sting

2019-01-07
Hornets tip off 6-game trip with loss to Western Conference-leading ...
Charlotte Observer

2019-01-05
Charlotte Hornets rookie Devonte Graham's journey from runt to ...
Charlotte Observer

